In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import sys
import os
BASE_DIR = os.path.abspath(os.path.join(os.path.dirname("__file__"), os.path.pardir))
sys.path.append(BASE_DIR)

In [ ]:
import cv2
import time
import numpy as np
import matplotlib.pyplot as plt
import imgaug as ia
import imgaug.augmenters as iaa
import tensorflow as tf

In [ ]:
from data_processor.data_loader import DataLoader, show_batch, DataLoaderWithoutCache
from models.dcgan import DCGAN, gen_random

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
cpus = tf.config.experimental.list_physical_devices(device_type='CPU')    
tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*7.5)])

In [ ]:
batch_size = 256
cache_size = 1024 * 64
nz = 100
glr = 2e-4
dlr = 2e-4
img_dir = 'data/faces/'
IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS = 64, 64, 3

def scale(img):
    return (img - 127.5) / 127.5

def rescale(img):
    return  img * 127.5 + 127.5

In [ ]:
sometimes = lambda aug: iaa.Sometimes(0.5, aug)

aug = iaa.Sequential(
    [
        iaa.Fliplr(0.5), # horizontally flip 50% of all images
        sometimes(iaa.CropAndPad(
            percent=(-0.05, 0.1),
            pad_mode=ia.ALL,
            pad_cval=(0, 255)
        )),
        sometimes(iaa.Affine(
            scale={"x": (0.9, 1.1), "y": (0.9, 1.1)}, # scale images to 80-120% of their size, individually per axis
            translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)}, # translate by -20 to +20 percent (per axis)
            rotate=(-10, 10), # rotate by -45 to +45 degrees
            order=[0, 1], # use nearest neighbour or bilinear interpolation (fast)
            cval=(0, 255), # if mode is constant, use a cval between 0 and 255
            mode=ia.ALL # use any of scikit-image's warping modes (see 2nd image from the top for examples)
        )),
    ],
    random_order=True
)

In [ ]:
data_loader = DataLoaderWithoutCache(data_dir=os.path.join(BASE_DIR, img_dir), img_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), cache_size=cache_size)
data_loader.scale(scale)\
            .batch(batch_size)\
            .augment(lambda x: aug(images=x))
img_batch = rescale(next(iter(data_loader)))
show_batch(img_batch)

In [ ]:
num_examples_to_generate = 36

seed = gen_random((num_examples_to_generate, nz))

In [ ]:
def show_generator(generator, seed):
    predictions = generator(seed, training=False).numpy()
    images = rescale(predictions).astype(np.uint8)
    show_batch(images)

In [ ]:
dcgan = DCGAN(image_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dlr=dlr, glr=glr, nz=nz)
dcgan.summary()

In [ ]:
show_generator(dcgan.generator, seed)

In [ ]:
for epoch in range(500):
    for batch_idx, img_batch in enumerate(data_loader):
        dcgan.train_step(img_batch, num_iter_disc=1, num_iter_gen=1)
        print(f'epoch: {epoch}, batch: {batch_idx}    ', end='\r')
    show_generator(dcgan.generator, seed)

In [ ]:
img_batch = rescale(next(iter(data_loader)))
show_batch(img_batch)

In [ ]:
show_generator(dcgan.generator, seed)